In [3]:
# --------------------------CODE BY MANDI DISHA--------------------------
import psycopg2
import pandas as pd
from psycopg2 import OperationalError

def connect_db():
    """Connect to the PostgreSQL database and return the connection and cursor."""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="postgres",  # Connected to the 'postgres' database
            user="postgres",
            port="5432",
            password="mandi"
        )
        print("Connection successful!")
        return conn, conn.cursor()
    except OperationalError as e:
        print(f"Connection failed: {e}")
        return None, None

def fetch_movie_data(cur):
    """Run the SQL query and return the results as a Pandas DataFrame."""
    query = """
        SELECT
        r.rating,  -- Grouping by movie rating
        COUNT(s.title) AS movie_count,
        ROUND(AVG(s.production_budget), 2) AS avg_production_budget,  -- Average production budget
        ROUND(AVG(m.userscore), 2) AS avg_userscore,  -- Average user score
        ROUND(AVG(s.worldwide_box_office), 2) AS avg_box_office,  -- Average worldwide box office
        ROUND((AVG(s.worldwide_box_office) - AVG(s.production_budget)) / AVG(s.production_budget) * 100, 2) AS avg_profit_percentage,  -- Profit percentage
        SUM(s.worldwide_box_office) AS total_box_office,  -- Total worldwide box office for each rating
        SUM(s.production_budget) AS total_budget,  -- Total budget for each rating
        ROUND(SUM(s.worldwide_box_office) / SUM(s.production_budget), 2) AS roi_ratio,  -- Return on Investment (ROI)
        
        -- Conditional profit calculation: Only include movies with a budget over $10 million
        ROUND(AVG(
            CASE 
                WHEN s.production_budget > 20000000 THEN (s.worldwide_box_office - s.production_budget) / s.production_budget * 100
                ELSE NULL
            END
        ), 2) AS avg_profit_large_budget_movies

    FROM
        movies m
    INNER JOIN
        sales s ON s.title = m.title
    AND
        s.release_year = EXTRACT(YEAR FROM m.reldate)
    LEFT JOIN 
        rating r ON m.url = r.url
    WHERE
        s.production_budget IS NOT NULL  -- Ensuring we only calculate for movies with known budgets
    AND
        m.userscore IS NOT NULL  -- Ensuring we only calculate for movies with known user scores
    GROUP BY
        r.rating  -- Grouping the results by rating
    ORDER BY
        avg_production_budget DESC,  -- Ordering by the average production budget (descending)
        avg_userscore DESC,  -- Ordering by the average user score (descending)
        avg_box_office DESC; 
    """
    cur.execute(query)
    # Fetch all rows from the query
    rows = cur.fetchall()
    
    # Get column names
    colnames = [desc[0] for desc in cur.description]

    # Return the data as a pandas DataFrame
    return pd.DataFrame(rows, columns=colnames)

def close_db(conn, cur):
    """Close the database connection and cursor."""
    cur.close()
    conn.close()

# Main function to encapsulate everything
def main():
    conn, cur = connect_db()
    if conn and cur:
        movie_data = fetch_movie_data(cur)
        print("Data from the query:")
        print(movie_data.head())  # Display the first few rows

        # Perform some analysis
        # Example: Describe the data
        print("\nSummary statistics:")
        print(movie_data.describe())

        # Example: Sort by average profit percentage
        sorted_data = movie_data.sort_values(by="avg_profit_percentage", ascending=False)
        print("\nTop ratings by profit percentage:")
        print(sorted_data.head())

        # Close the database connection
        close_db(conn, cur)

if __name__ == "__main__":
    main()


#  [END CODE BY MANDI DISHA]





Connection successful!
Data from the query:
    rating  movie_count avg_production_budget avg_userscore avg_box_office  \
0     TV-G            4           91500000.00          7.03   473407170.25   
1        G           48           70885416.67          6.73   245814300.02   
2       PG          428           64224567.76          6.43   201635396.44   
3    PG-13         1122           57289148.87          6.46   171838831.59   
4  Unrated           65           37464924.34          6.48   106835608.08   

  avg_profit_percentage total_box_office total_budget roi_ratio  \
0                417.38       1893628681    366000000      5.17   
1                246.78      11799086401   3402500000      3.47   
2                213.95      86299949677  27488115000      3.14   
3                199.95     192803169045  64278425029      3.00   
4                185.16       6944314525   2435220082      2.85   

  avg_profit_large_budget_movies  
0                         350.00  
1             

In [4]:
#Done by Anastasiya 
#This code connects to a PostgreSQL database, cleans up movie data 
#by removing records with missing values (e.g., for production budget or worldwide box office), 
#and then runs an SQL query to analyze movie performance metrics. 
#The query retrieves the average production budget, metascore, 
#and worldwide box office grouped by the movie's release month. 
#The results are fetched and loaded into a Pandas DataFrame for further analysis,
#including generating summary statistics and sorting movies by production budget. 
#The script then commits any changes and closes the database connection.
import psycopg2
import pandas as pd
from psycopg2 import OperationalError

def connect_db():
    """Connect to the PostgreSQL database and return the connection and cursor."""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="postgres",  # Connected to the 'postgres' database
            user="postgres",
            port="5432",
            password="Foundation.23"
        )
        print("Connection successful!")
        return conn, conn.cursor()
    except OperationalError as e:
        print(f"Connection failed: {e}")
        return None, None

def fetch_movie_data(cur):
    """Run the SQL query and return the results as a Pandas DataFrame."""
    query = """
    SELECT 
        EXTRACT(MONTH FROM m.reldate) AS release_month,  -- Extracting the month from the release date
        ROUND(AVG(s.production_budget), 2) AS avg_production_budget,  -- Average production budget
        ROUND(AVG(m.metascore), 2) AS avg_metascore,  -- Average metascore
        ROUND(AVG(s.worldwide_box_office), 2) AS avg_box_office  -- Average worldwide box office
    FROM
        sales_v20 s
    INNER JOIN
        movies2 m
    ON
        s.title = m.title
    AND
        s.release_year = EXTRACT(YEAR FROM m.reldate)
    WHERE
        s.production_budget IS NOT NULL  -- Only calculate for movies with known budgets
    AND
        m.metascore IS NOT NULL  -- Only calculate for movies with known metascores
    GROUP BY
        EXTRACT(MONTH FROM m.reldate)  -- Grouping by the month
    ORDER BY
        avg_production_budget DESC,  -- Then by production budget (descending)
        avg_metascore,
        release_month;
    """
    cur.execute(query)
    
    # Fetch all rows from the query
    rows = cur.fetchall()
    
    # Get column names
    colnames = [desc[0] for desc in cur.description]

    # Return the data as a pandas DataFrame
    return pd.DataFrame(rows, columns=colnames)

def clean_data(cur):
    """Delete rows with NULL values in production_budget or worldwide_box_office."""
    delete_query = """
    DELETE FROM sales_v20
    WHERE production_budget IS NULL
    OR worldwide_box_office IS NULL;
    """
    cur.execute(delete_query)


def close_db(conn, cur):
    """Close the database connection and cursor."""
    cur.close()
    conn.close()

# Main function to encapsulate everything
def main():
    conn, cur = connect_db()
    if conn and cur:
        # Clean the data first by removing null/empty values
        clean_data(cur)

        # Fetch the movie data after cleaning
        movie_data = fetch_movie_data(cur)
        print("Data from the query:")
        print(movie_data.head())  # Display the first few rows

        # Perform some analysis
        # Example: Describe the data
        print("\nSummary statistics:")
        print(movie_data.describe())

        # Example: Sort by 'avg_production_budget' (since 'avg_profit_percentage' doesn't exist)
        sorted_data = movie_data.sort_values(by="avg_production_budget", ascending=False)
        print("\nTop movies by production budget:")
        print(sorted_data.head())

        # Commit any changes (in case of DELETE) and close the database connection
        conn.commit()
        close_db(conn, cur)

# Fix the main function entry point
if __name__ == "__main__":
    main()

Connection successful!
Data from the query:
  release_month avg_production_budget avg_metascore avg_box_office
0             5           63597304.48         56.15   205076656.27
1             6           59944112.86         56.54   195103356.17
2             7           55723245.93         55.85   177124325.11
3            11           53939179.75         59.77   160756398.24
4            12           51481538.46         60.52   163654119.23

Summary statistics:
       release_month avg_production_budget avg_metascore avg_box_office
count             12                    12            12             12
unique            12                    12            12             12
top                5           63597304.48         56.15   205076656.27
freq               1                     1             1              1

Top movies by production budget:
  release_month avg_production_budget avg_metascore avg_box_office
0             5           63597304.48         56.15   205076656.27
1    